In [1]:
import pandas as pd
import requests

In [289]:
API_KEY = '881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36'

In [291]:
next_url = "https://api.agendor.com.br/v3/deals"

# Cabeçalhos da requisição
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

params={"per_page": 100}

all_json_deals_data = []

while next_url:
    response = requests.get(next_url, headers=headers, params=params)
    json_deal_data = response.json()['data']
    all_json_deals_data.extend(json_deal_data)
    links = response.json()['links']
    if 'next' in links.keys():
        next_url = links['next']
    else:
        next_url = False




In [293]:
import json

# Salvando como JSON
with open("deals.json", "w", encoding="utf-8") as arquivo:
    json.dump(all_json_deals_data, arquivo, indent=4, ensure_ascii=False)

In [2]:
import json

# Ler arquivo JSON com vários objetos
with open("deals.json", "r", encoding="utf-8") as arquivo:
    deals = json.load(arquivo)

In [9]:
titles = {'title': []}
for deal in deals:
    titles['title'].append(deal['title'])

In [25]:
df = pd.DataFrame(titles)

In [39]:
deals[:30]

[{'id': 29068938,
  'accountId': 698720,
  'organization': None,
  'person': {'id': 55574570, 'name': 'Andre Dias De Matos', 'email': None},
  'author': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'dealStage': {'id': 3186643,
   'name': '2.1 VALIDAÇÃO',
   'sequence': 1,
   'funnel': {'id': 773196, 'name': '2 SERVICE'}},
  'dealStatus': {'id': 2, 'name': 'Ganho'},
  'lossReason': None,
  'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'title': 'Andre Dias de Matos - 02 ',
  'description': None,
  'startTime': '2025-01-24T13:33:30.000Z',
  'endTime': '2025-02-04T00:00:00.000Z',
  'wonAt': '2025-02-04T20:33:52.000Z',
  'lostAt': None,
  'createdAt': '2025-01-24T13:33:30.000Z',
  'updatedAt': '2025-02-04T20:33:52.000Z',
  'ranking': 0,
  'value': 0.0,
  'products': [],
  'allowedUsers': [{'accountId': 698720,
    'id': 860639,
    'name': 'Guilherme Hely'},
   {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
   {'accountId': 698720, '

In [85]:
pd.DataFrame(deal_by_stage).to_csv('deal_by_stage.csv', index=False, encoding='utf-8')

In [83]:
deal_by_stage = {
    'stage_detail': [], 'stage_name': [], 'stage_number': [], 'stage_status': [],
    'person': [], 'title': [], 'date_created': [], 'date_lost': [], 'date_won': [],
    'organization': []
}
for deal in deals:
    stage_detail = deal['dealStage']['name']
    stage_number = int(deal['dealStage']['sequence'])
    stage_name = deal['dealStage']['funnel']['name']
    stage_status = deal['dealStatus']['name']
    if deal['person']:
        person = deal['person']['id']
    else:
        person = None
    if deal['organization']:
        organization = deal['organization']['id']
    else:
        organization = None
    date_created = deal['createdAt']
    date_won = deal['wonAt']
    date_lost = deal['lostAt']
    title = deal['title']
    deal_by_stage['stage_detail'].append(stage_detail)
    deal_by_stage['stage_number'].append(stage_number)
    deal_by_stage['stage_name'].append(stage_name)
    deal_by_stage['stage_status'].append(stage_status)
    deal_by_stage['person'].append(person)
    deal_by_stage['title'].append(title)
    deal_by_stage['date_created'].append(date_created)
    deal_by_stage['date_lost'].append(date_lost)
    deal_by_stage['date_won'].append(date_won)
    deal_by_stage['organization'].append(organization)

    

In [137]:
from datetime import datetime, timedelta

In [173]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
from datetime import datetime, timedelta

In [191]:
deals[:40]

[{'id': 29068938,
  'accountId': 698720,
  'organization': None,
  'person': {'id': 55574570, 'name': 'Andre Dias De Matos', 'email': None},
  'author': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'dealStage': {'id': 3186643,
   'name': '2.1 VALIDAÇÃO',
   'sequence': 1,
   'funnel': {'id': 773196, 'name': '2 SERVICE'}},
  'dealStatus': {'id': 2, 'name': 'Ganho'},
  'lossReason': None,
  'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'title': 'Andre Dias de Matos - 02 ',
  'description': None,
  'startTime': '2025-01-24T13:33:30.000Z',
  'endTime': '2025-02-04T00:00:00.000Z',
  'wonAt': '2025-02-04T20:33:52.000Z',
  'lostAt': None,
  'createdAt': '2025-01-24T13:33:30.000Z',
  'updatedAt': '2025-02-04T20:33:52.000Z',
  'ranking': 0,
  'value': 0.0,
  'products': [],
  'allowedUsers': [{'accountId': 698720,
    'id': 860639,
    'name': 'Guilherme Hely'},
   {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
   {'accountId': 698720, '

In [187]:
df = pd.DataFrame(deal_by_stage)


# Supondo que df já esteja carregado
# Criar as colunas 'id' e 'type'
df['id'] = df['person'].fillna(df['organization'])
df['type'] = df['person'].apply(lambda x: 'person' if pd.notna(x) else 'organization')

# Remover as colunas 'person' e 'organization'
df = df.drop(columns=['person', 'organization'])

# Remover números no início da coluna 'stage_name'
df['stage_name'] = df['stage_name'].str.replace(r'^\d+\s*', '', regex=True)

# Filtragem por data
df['date_created'] = pd.to_datetime(df['date_created']).dt.date

# Transformação da coluna 'stage_detail'
df['stage_detail'] = df['stage_detail'].replace({
    'CONTATO': '1.1 LEADS',
    'TYPEFORM': '2.1 VALIDAÇÃO',
    'CONTRATO': '3.1 ATIVOS'
})

# Link externo para CSS
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Criar o aplicativo Dash
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Dashboard de Stage Detail"

# Valor inicial do filtro de data
default_start_date = datetime.today() - timedelta(days=60)
default_end_date = datetime.today()

# Layout do dashboard
app.layout = html.Div(children=[
    html.H1(children="Dashboard de Stage Detail", style={'text-align': 'center', 'color': '#003366'}),

    # Container para os filtros dinâmicos
    html.Div(id='date-filters-container', children=[
        html.Div([
            html.Label(f'Filtro 1', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': 0},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
    ]),

    # Botão para adicionar filtros
    html.Button("Adicionar Filtro", id="add-filter-btn", n_clicks=0, style={'background-color': '#003366', 'color': 'white'}),

    # Gráfico
    dcc.Graph(id='line-chart')
], style={'font-family': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#FFFFFF'})

# Callback para adicionar novos filtros dinamicamente
@app.callback(
    Output('date-filters-container', 'children'),
    Input('add-filter-btn', 'n_clicks'),
    State('date-filters-container', 'children')
)
def add_date_filter(n_clicks, children):
    if n_clicks > 0:
        new_filter_index = n_clicks + 1
        new_filter = html.Div([
            html.Label(f'Filtro {new_filter_index}', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': n_clicks},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
        children.append(new_filter)
    return children

# Callback para atualizar o gráfico baseado nos filtros aplicados
@app.callback(
    Output('line-chart', 'figure'),
    [Input({'type': 'date-filter', 'index': dash.ALL}, 'start_date'),
     Input({'type': 'date-filter', 'index': dash.ALL}, 'end_date')]
)
def update_chart(start_dates, end_dates):
    fig = px.line(title="Evolução por Stage Detail")
    if not start_dates or not end_dates:
        return fig

    for i, (start_date, end_date) in enumerate(zip(start_dates, end_dates)):
        if start_date and end_date:
            start_date = datetime.fromisoformat(start_date).date()
            end_date = datetime.fromisoformat(end_date).date()
            filtered_df = df[(df['date_created'] >= start_date) & (df['date_created'] <= end_date)]
            filtered_df = filtered_df.groupby('stage_detail').size().reset_index(name='total')
            fig.add_scatter(x=filtered_df['stage_detail'], y=filtered_df['total'], mode='lines+markers', name=f'Filtro {i + 1}')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font={'color': '#003366'})
    return fig

# Rodar o app
if __name__ == '__main__':
    app.run_server(debug=True)